In [ ]:
import os

import cv2
import numpy as np
import scipy

import tensorflow as tf

# 0. Настройка основных параметров

In [ ]:
# Main folder path
PROJECT_FOLDER = '/content/drive/MyDrive/AI/IVision'

In [ ]:
# Other folders
DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'datasets')

MODEL_FOLDER = os.path.join(PROJECT_FOLDER, 'model')
MODEL_PATH = os.path.join(MODEL_FOLDER, 'ModelTF.h5')

In [ ]:
# Model params
BATCH_SIZE = 100
IMAGE_SIZE = (250, 250)  # im_height, im_width

In [ ]:
VIDEO_PATH = os.path.join(DATA_FOLDER, '1.mp4')

# 1. Загрузка модели

In [ ]:
MyCnn = tf.keras.models.load_model(
    MODEL_PATH, custom_objects=None, compile=True, options=None
)

OSError: ignored

# 2. Скоринг видео

In [ ]:
def predict_video(model, video_path, th=0.011):

  cap = cv2.VideoCapture(video_path)
  batch = []
  probs_0 = []  # accident
  probs_1 = []  # no accident

  while(cap.isOpened()):
      ret, frame = cap.read()
      if len(batch) < BATCH_SIZE and ret == True:
        batch.append(cv2.resize(frame, IMAGE_SIZE))

      elif len(batch) >= BATCH_SIZE or ret == False:
        batch = tf.convert_to_tensor(batch, dtype=None, dtype_hint=None, name=None)
        predictions = model.predict(batch)

        probs_0.append(predictions[:,0])
        probs_1.append(predictions[:,1])

        batch = []

      if ret == False:
        break
  
  cap.release()

  probs_0, probs_1 = np.concatenate(probs_0), np.concatenate(probs_1)
  evr = probs_0[probs_0 >= scipy.stats.mode(probs_0)[0][0]].std()

  if evr >= th:
    print('Авария!')
  
  else:
    print('Нет аварии!')
  
  return

In [ ]:
predict_video(MyCnn, VIDEO_PATH)

Авария!
